#### Import packages

In [ ]:
import pandas as pd
import numpy as np
import pyodbc as py

from datetime import date, timedelta, datetime
import time
import math

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn import preprocessing
from sklearn.utils import class_weight
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

from itertools import product

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll.base import scope
from time import time

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils

import json
from numpyencoder import NumpyEncoder

from dynamic_input_data import *
import importlib
importlib.reload(dynamic_input_data)

import warnings
import random
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)

#### Load Data

In [ ]:
def initialiseData():
    
    # Read in cleaned and prepared data file (.csv) that is created in the data_cleaning_preparation code
    df = pd.read_csv('path...', low_memory = True)
    
    df['orderDate']                   = pd.to_datetime(df['orderDate'])
    df['cancellationDate']            = pd.to_datetime(df['cancellationDate'])
    df['promisedDeliveryDate']        = pd.to_datetime(df['promisedDeliveryDate'])
    df['shipmentDate']                = pd.to_datetime(df['shipmentDate'])
    df['dateTimeFirstDeliveryMoment'] = pd.to_datetime(df['dateTimeFirstDeliveryMoment'])
    df['startDateCase']               = pd.to_datetime(df['startDateCase'])
    df['returnDateTime']              = pd.to_datetime(df['returnDateTime'])
    df['registrationDateSeller']      = pd.to_datetime(df['registrationDateSeller'])

    # Fixed Columns:
    DATE = ['orderDate']
    BASIC = ['totalPrice','quantityOrdered','fulfilmentByPlatform','countryCodeNL','countryOriginNL','countryOriginBE',
            'countryOriginDE','productTitleLength','promisedDeliveryDays','partnerSellingDays', 'orderCorona']
    WEEK = ['orderMonday','orderTuesday','orderWednesday','orderThursday','orderFriday','orderSaturday','orderSunday']
    MONTH = ['orderJanuary','orderFebruary','orderMarch','orderApril','orderMay','orderJune',
             'orderJuly','orderAugust','orderSeptember','orderOctober','orderNovember','orderDecember']
    YEAR = ['orderYear2020']
    GROUP = ['groupHealth','groupHome','groupSports','groupComputer','groupPets','groupToys','groupBooks', 
             'groupBaby', 'groupMusic', 'groupFood','groupOffice','groupFashion','groupOther','groupCar']

    # Dynamic Columns:
    TRANSPORTERX = ['transporterPOSTNL/X','transporterDHL/X','transporterDPD/X','transporterBRIEF/X','transporterOTHER/X']
    KNOWNX = ['caseKnownX','returnKnownX','cancellationKnownX','onTimeDeliveryKnownX','lateDeliveryKnownX']
    PRODUCTX = ['productOrderCountX','productTotalCountX','productTotalReturnedX','productReturnFractionX']
    SELLERX = ['sellerDailyOrdersX']
    HISTORICX = []
    historic_variable = ['transporterCode','sellerId','productGroup']
    for x in range(len(historic_variable)):
        HISTORICX = HISTORICX + [historic_variable[x]+'HistoricHappyX',historic_variable[x]+'HistoricUnhappyX',historic_variable[x]+'HistoricUnknownX']

    # Determinants:
    DETERMINANT = ['noReturn', 'noCase', 'noCancellation', 'onTimeDelivery']

    # Classifications
    CLASSIFICATION = ['generalMatchClassification','detailedMatchClassification','binaryMatchClassification','determinantClassification']

    X_col = BASIC + WEEK + MONTH + YEAR + GROUP + TRANSPORTERX + KNOWNX + PRODUCTX + SELLERX + HISTORICX
    Y_col = ['detailedMatchClassification']
    
    return df, X_col, Y_col, DATE, historic_variable

In [ ]:
# Initialise data
df, X_col, Y_col, DATE, historic_variable = initialiseData()

#### Validation (Flat)

In [ ]:
# Fill dataframe df_ with a subsample of training + validation data containing 1.1M observations
random.seed(100)
df_ = df.iloc[:int(0.8*len(df))].sample(n=1100000, replace=False, random_state=1).sort_values(by = 'orderDate').reset_index(drop = True)

In [ ]:
def flat_get_hyperspace(combination):
    
    param_hyperopt = {}

    if combination == 'DT':
        hyper = {'DT_criterion'   : hp.choice('DT_criterion',['gini','entropy']),
                 'DT_max_depth'   : scope.int(hp.quniform('DT_max_depth', 5, 15, 1))}
    elif combination == 'RF':
        hyper = {'RF_max_depth'    : scope.int(hp.quniform('RF_max_depth', 5, 15, 1)),
                 'RF_n_estimators' : scope.int(hp.quniform('RF_n_estimators', 10, 50, 5))}
    elif combination == 'NN':
        hyper = {'NN_dropout'  : hp.uniform('NN_dropout', 0, 0.5),
                 'NN_nodes'    : scope.int(hp.quniform('NN_nodes', 5, 50, 5)),
                 'NN_layers'   : scope.int(hp.quniform('NN_layers', 1, 2, 1))}
    elif combination == 'LR':
        hyper = {'LR_penalty' : hp.choice('LR_penalty', ['l1','l2'])}

    param_hyperopt = {**param_hyperopt, **hyper}
        
    return param_hyperopt

In [ ]:
def flat_objective_function(params):
    
    if combination == 'RF':
        clf = RandomForestClassifier(random_state=0, class_weight='balanced', max_depth = params['RF_max_depth'], n_estimators = params['RF_n_estimators'])
    elif combination == 'LR':
        print(params)
        clf = LogisticRegression(penalty = params['LR_penalty'], class_weight = 'balanced', solver = 'liblinear')
    
    clf = clf.fit(X_train,y_train)
    pred = clf.predict(X_val)
    
    precision, recall, f1, support = metrics.precision_recall_fscore_support(y_val, pred, average = 'weighted', beta = 1)
    accuracy = metrics.accuracy_score(y_val, pred)
    #cross validation score to be implemented?
    
    return {'loss': -f1, 'status': STATUS_OK, 'accuracy': accuracy}

In [ ]:
def flat_hyperopt(param_space, X_train, y_train, X_val, y_val, num_eval):

    trials = Trials()

    best_param = fmin(flat_objective_function, 
                      param_space, 
                      algo = tpe.suggest, 
                      max_evals = num_eval, 
                      trials = trials,
                      rstate = np.random.RandomState(1))
    
    loss = [x['result']['loss'] for x in trials.trials]
    index_min_loss = loss.index(min(loss))
    accuracy_scores = [x['result']['accuracy'] for x in trials.trials]
    
    f1 = min(loss)*-1
    accuracy = accuracy_scores[index_min_loss]
    
    return best_param, f1, accuracy

In [ ]:
# Set start day as date of ordering (begin = 0) and end day as the 10-th day after the order date (end = 10)
begin, end = 0, 10

In [ ]:
# Initialise output
output = {}

# All combinations between RF and LR in the flat case
combinations = ['RF','LR']

# For all days
for DAY in range(begin,end+1):
    
    # Create X and Y 
    X_preBurn, y_preBurn = dataX(df_, DATE, X_col, Y_col, historic_variable, DAY)
    index = range(0, X_preBurn.shape[0])
    
    # Burn in first 10% of observations
    X_train_val = X_preBurn.iloc[int(0.1*len(X_preBurn)):]
    y_train_val = y_preBurn.iloc[int(0.1*len(y_preBurn)):]

    # Make 80-20 train-validation split
    X_train = X_train_val.iloc[0:int(0.8*len(X_train_val))]
    X_val = X_train_val.iloc[int(0.8*len(X_train_val)):]

    y_train = y_train_val.iloc[0:int(0.8*len(y_train_val))]
    y_val = y_train_val.iloc[int(0.8*len(y_train_val)):]
    
    output[DAY] = {}

    for combination in combinations:
        
        if combination == 'RF':
            n_trials = 20
        elif combination == 'LR':
            n_trials = 2 # No more than 2 trials needed to try all hyperparameter combinations (we only tune on either L1- or L2-penalty)

        # Optimise
        best_param, f1, accuracy = flat_hyperopt(flat_get_hyperspace(combination), X_train, y_train, X_val, y_val, n_trials)

        # Save output for day and combination
        output[DAY][str(combination)] = (DAY, best_param, f1, accuracy)
        print(output)
        
        # Write a path and file name (with .json format) to write the output to
        with open('path...', 'w') as f:
            json.dump(output, f, cls = NumpyEncoder)

#### Validation (HCOT)

In [ ]:
# Fill dataframe df_ with a subsample of training + validation data containing 1.1M observations
random.seed(100)
df_ = df.iloc[:int(0.8*len(df))].sample(n=1100000, replace=False, random_state=1).sort_values(by = 'orderDate').reset_index(drop = True)

In [ ]:
# Create hierarchical structure
Tree = ClassHierarchy('ORDERS')
Tree.add_node(['UNKNOWN','KNOWN'], 'ORDERS')
Tree.add_node(['HAPPY','UNHAPPY'], 'KNOWN')
Tree.add_node(['MILDLY UNHAPPY','MEDIUM UNHAPPY','HEAVILY UNHAPPY'], 'UNHAPPY')

## All combinations for RF and LR
combinations = [('LR','LR','LR'),('LR','LR','RF'),('LR','RF','LR'),('LR','RF','RF'),
                ('RF','LR','LR'),('RF','LR','RF'),('RF','RF','LR'),('RF','RF','RF')]

In [ ]:
# Initialise
output = {}
START = 0
END = 10

for DAY in range(START,END+1):
    
    # Create X and Y 
    X_preBurn, y_preBurn = dataX(df_, DATE, X_col, Y_col, historic_variable, DAY)
    index = range(0, X_preBurn.shape[0])
    
    # Burn in first 10% of observations
    X_train_val = X_preBurn.iloc[int(0.1*len(X_preBurn)):]
    y_train_val = y_preBurn.iloc[int(0.1*len(y_preBurn)):]

    # Make 80-20 train-validation split
    X_train = X_train_val.iloc[0:int(0.8*len(X_train_val))]
    X_val = X_train_val.iloc[int(0.8*len(X_train_val)):]

    y_train = y_train_val.iloc[0:int(0.8*len(y_train_val))]
    y_val = y_train_val.iloc[int(0.8*len(y_train_val)):]

    output[DAY] = {}

    for combination in combinations:

        # Optimise
        best_param, f1, accuracy = hyperopt(get_hyperspace(combination), X_train, y_train, X_val, y_val, 20)

        # Save output per combination per day
        output[DAY][str(combination)] = (DAY, best_param, f1, accuracy)

        # Write a path and file name (with .json format) to write the output to
        with open('path...', 'w') as f:
            json.dump(output, f, cls = NumpyEncoder)
    
    print(DAY)

In [ ]:
def get_hyperspace(combination):
    
    param_hyperopt = {}
    
    for node, clf in enumerate(combination):
        
        if clf == 'DT':
            hyper = {'DT_criterion_'+str(node)   : hp.choice('DT_criterion_'+str(node) ,['gini','entropy']),
                     'DT_max_depth_'+str(node)   : scope.int(hp.quniform('DT_max_depth_'+str(node), 5, 15, 1))}
        elif clf == 'RF':
            hyper = {'RF_max_depth_'   +str(node) : scope.int(hp.quniform('RF_max_depth_'+str(node), 5, 15, 1)),
                     'RF_n_estimators_'+str(node) : scope.int(hp.quniform('RF_n_estimators_'+str(node), 10, 50, 5))}
        elif clf == 'NN':
            hyper = {'NN_dropout_'+str(node)  : hp.uniform('NN_dropout_'+str(node), 0, 0.5),
                     'NN_nodes_'  +str(node)  : scope.int(hp.quniform('NN_nodes_'+str(node), 5, 50, 5)),
                     'NN_layers_' +str(node)  : scope.int(hp.quniform('NN_layers_'+str(node), 1, 2, 1))}
        elif clf == 'LR':
            hyper = {'LR_penalty_' + str(node) : hp.choice('LR_penalty_' + str(node), ['l1','l2'])}
            
        param_hyperopt = {**param_hyperopt, **hyper}
        
    return param_hyperopt

In [ ]:
def clf_hypers(params):

    clf = {}
    
    for ix, node in enumerate(['ORDERS','KNOWN','UNHAPPY']):

        node_hypers = [x for x in list(params.keys()) if x[-1] == str(ix)]

        if combination[ix] == 'DT':
            clf[node] = DecisionTreeClassifier(random_state=0, class_weight='balanced', max_depth = params[node_hypers[1]], criterion = params[node_hypers[0]])
        elif combination[ix] == 'RF':
            clf[node] = RandomForestClassifier(random_state=0, class_weight='balanced', max_depth = params[node_hypers[0]], n_estimators = params[node_hypers[1]])
        elif combination[ix] == 'NN':
            if ix == 2:
                output = 3
            else:
                output = 1
            clf[node] = KerasClassifier(functions.neuralNetwork, output = output, nodes = params[node_hypers[1]], layers = params[node_hypers[2]], droprate = params[node_hypers[0]], epochs = 15, verbose = 0)
        elif combination[ix] == 'LR':
            clf[node] = LogisticRegression(penalty = params[node_hypers[0]], class_weight = 'balanced', solver = 'liblinear')
            
    return clf

In [ ]:
def objective_function(params):
    
    HC = HierarchicalClassifier(Tree)
    HC.fit_classifiers(clf_hypers(params))
    
    HC = HC.fit(X_train,y_train)
    pred = HC.predict(X_val)
    
    score = f1_score_ancestors(Tree, y_val['detailedMatchClassification'], pred, beta=1)
    accuracy = metrics.accuracy_score(y_val, pred)
    # cross-validation score to be implemented
    
    return {'loss': -score, 'status': STATUS_OK, 'accuracy': accuracy}

In [ ]:
def hyperopt(param_space, X_train, y_train, X_val, y_val, num_eval):

    trials = Trials()

    best_param = fmin(objective_function, 
                      param_space, 
                      algo = tpe.suggest, 
                      max_evals = num_eval, 
                      trials = trials,
                      rstate = np.random.RandomState(1))
    
    loss = [x['result']['loss'] for x in trials.trials]
    index_min_loss = loss.index(min(loss))
    accuracy_scores = [x['result']['accuracy'] for x in trials.trials]
    
    f1 = min(loss)*-1
    accuracy = accuracy_scores[index_min_loss]
    
    return best_param, f1, accuracy

#### CAT-HCOT

In [ ]:
def dynamicHierarchicalClassifier(START, END, threshold = None, threshold_type = None):  
  
    # Create hierarchy
    Tree = ClassHierarchy('ORDERS')
    Tree.add_node(['UNKNOWN','KNOWN'], 'ORDERS')
    Tree.add_node(['HAPPY','UNHAPPY'], 'KNOWN')
    Tree.add_node(['MILDLY UNHAPPY','MEDIUM UNHAPPY','HEAVILY UNHAPPY'], 'UNHAPPY')
    
    # Optimal hypers obtained via validation
    hypers = pd.DataFrame({'1_penalty'     : ['l1','l1','l2','l2','l2','l2','l1','l1','l1','l1','l1'],
                           '2_max_depth'   : [ 9,10,12,12,12,12,10,10,10,10,10], 
                           '2_n_estimators': [35,45,30,30,30,30,45,45,45,45,45],
                           '3_max_depth'   : [14,14,14,14,14,14,14,14,14,14,14], 
                           '3_n_estimators': [20,45,30,30,30,30,45,45,45,45,45]})
    
    # Certainty parameter alpha
    CERTAINTY = 0.7
    OPTION = 2
    
    statistics, previous_pred_block, feature_importances = None, None, None

    for DAYS in range(START, END+1):
        
        # Create X and Y
        X, y = dataX(df, DATE, X_col, Y_col, historic_variable, DAYS)

        # Make 80-20 train-test split and burn in first 10% of train set
        X_train_preburn = X.iloc[:int(0.8*len(X))]
        y_train_preburn = y.iloc[:int(0.8*len(y))]

        X_train = X_train_preburn.iloc[int(0.1*len(X_train_preburn)):]
        y_train = y_train_preburn.iloc[int(0.1*len(y_train_preburn)):]

        X_test = X.iloc[int(0.8*len(X)):]
        y_test = y.iloc[int(0.8*len(y)):]

        N_test = len(y_test)
    
        # Fit classifiers at parent nodes in hierarchy
        HC = HierarchicalClassifier(Tree)
        HC.fit_classifiers({'ORDERS'  : LogisticRegression(random_state=0, class_weight='balanced', solver = 'liblinear', penalty = hypers.loc[DAYS, '1_penalty']),
                            'KNOWN'   : RandomForestClassifier(random_state=0, class_weight='balanced', max_depth = hypers.loc[DAYS, '2_max_depth'], n_estimators = hypers.loc[DAYS, '2_n_estimators']),
                            'UNHAPPY' : RandomForestClassifier(random_state=0, class_weight='balanced', max_depth = hypers.loc[DAYS, '3_max_depth'], n_estimators = hypers.loc[DAYS, '3_n_estimators'])})

        HC = HC.fit(X_train,y_train)

        y_train_hat = HC.get_probabilities(X_train, y_train)
        probs = pd.concat([y_train, y_train_hat], axis=1)

        # Find thresholds
        THRESHOLDS = {}
        for node in range(1,8):
            name, threshold = opt_threshold(probs, node, DAYS, CERTAINTY, option = 2)
            THRESHOLDS[name] = threshold

        if DAYS == START: # create dataframe to save predictions
            y_hat = pd.DataFrame([Tree.root] * len(X_test),
                                    columns=[DAYS],
                                    index=X_test.index)
            index_no_leaf = X_test.index
        else:
            y_hat[DAYS] = y_hat[DAYS - 1]

        if DAYS < END:
            pred = HC.predict_proba2(X_test.loc[index_no_leaf], THRESHOLDS = THRESHOLDS)

            check_no_leaf = ~pred.isin(Tree._get_leaf_nodes())
            index_no_leaf = check_no_leaf[check_no_leaf].index
            check_leaf    = pred.isin(Tree._get_leaf_nodes())      # from current non_leaf predictions which are now leaf
            index_leaf    = check_leaf[check_leaf].index
            y_hat_stage   = pd.DataFrame(pred, index = index_leaf)
        else:
            pred        = HC.predict(X_test.loc[index_no_leaf]) # last day you want a label for each order
            y_hat_stage = pd.DataFrame(pred, index = index_no_leaf)
            index_leaf  = index_no_leaf

        y_hat = y_hat.assign(stage_col = y_hat_stage)
        y_hat.stage_col = y_hat.stage_col.fillna(y_hat[DAYS]) #  fill previously predicted labels
        y_hat = y_hat.drop(DAYS, axis=1)
        y_hat = y_hat.rename(columns={'stage_col': DAYS})

        current_pred = y_hat.iloc[:, y_hat.shape[1] - 1]
        
        # Save performance metrics
        statistics, feature_importances, previous_pred_block = get_performance(DAYS, END, pred, current_pred, index_leaf, index_no_leaf, 
                                                                               previous_pred_block, THRESHOLDS, OPTION, CERTAINTY, y_test, Tree, HC, feature_importances, statistics)

        # Write a path and file name (with .json format) to write the output to
        file_name = 'statistics_optimal_'+str(OPTION)+'_'+str(CERTAINTY)+'.json'
        path_name = 'path...' + file_name
        with open(path_name, 'w') as f:
            json.dump(statistics, f, cls = NumpyEncoder)

        print('DAYS: ',DAYS)
     
    final_pred = y_hat.iloc[:, y_hat.shape[1] - 1]
        
    return final_pred, statistics, feature_importances

In [ ]:
# Run CAT-HCOT 
pred, statistics, feature_importances = dynamicHierarchicalClassifier(0,10)

#### Base Case 1 (Static)

In [ ]:
def staticHierarchicalClassifier(START, END):
    
    # Create hierarchy
    Tree = ClassHierarchy('ORDERS')
    Tree.add_node(['UNKNOWN','KNOWN'], 'ORDERS')
    Tree.add_node(['HAPPY','UNHAPPY'], 'KNOWN')
    Tree.add_node(['MILDLY UNHAPPY','MEDIUM UNHAPPY','HEAVILY UNHAPPY'], 'UNHAPPY')
    
    # Optimal hypers obtained via validation
    hypers = pd.DataFrame({'1_penalty'     : ['l1','l1','l2','l2','l2','l2','l1','l1','l1','l1','l1'],
                           '2_max_depth'   : [ 9,10,12,12,12,12,10,10,10,10,10], 
                           '2_n_estimators': [35,45,30,30,30,30,45,45,45,45,45],
                           '3_max_depth'   : [14,14,14,14,14,14,14,14,14,14,14], 
                           '3_n_estimators': [20,45,30,30,30,30,45,45,45,45,45]})
    
    statistics = {'accuracy':{},
                  'precision':{},
                  'recall':{},
                  'f1':{}}
    for leaf in Tree._get_leaf_nodes(): 
        statistics['precision_'+leaf] = {}
        statistics['recall_'+leaf]    = {}
        statistics['f1_'+leaf]        = {}
    
    for DAYS in range(START, END+1):

        # Create X and Y
        X, y = dataX(df, DATE, X_col, Y_col, historic_variable, DAYS)

        # Make 80-20 train-test split and burn in first 10% of train set
        X_train_preburn = X.iloc[:int(0.8*len(X))]
        y_train_preburn = y.iloc[:int(0.8*len(y))]

        X_train = X_train_preburn.iloc[int(0.1*len(X_train_preburn)):]
        y_train = y_train_preburn.iloc[int(0.1*len(y_train_preburn)):]

        X_test = X.iloc[int(0.8*len(X)):]
        y_test = y.iloc[int(0.8*len(y)):]

        # Fit classifiers at parent nodes in hierarchy
        HC = HierarchicalClassifier(Tree)
        HC.fit_classifiers({'ORDERS'  : LogisticRegression(random_state=0, class_weight='balanced', solver = 'liblinear', penalty = hypers.loc[DAYS, '1_penalty']),
                            'KNOWN'   : RandomForestClassifier(random_state=0, class_weight='balanced', max_depth = hypers.loc[DAYS, '2_max_depth'], n_estimators = hypers.loc[DAYS, '2_n_estimators']),
                            'UNHAPPY' : RandomForestClassifier(random_state=0, class_weight='balanced', max_depth = hypers.loc[DAYS, '3_max_depth'], n_estimators = hypers.loc[DAYS, '3_n_estimators'])})

        HC = HC.fit(X_train,y_train)
        pred = HC.predict(X_test)
        
        y_test = y_test['detailedMatchClassification']
        
        # Compute performance metrics
        statistics['accuracy'][DAYS] = metrics.accuracy_score(y_test, pred)
        statistics['precision'][DAYS] = precision_score_ancestors(Tree, y_test, pred)
        statistics['recall'][DAYS] = recall_score_ancestors(Tree, y_test, pred)
        statistics['f1'][DAYS] = f1_score_ancestors(Tree, y_test, pred, beta = 1)
        
        for leaf in Tree._get_leaf_nodes():
            leaf_ix = pred.loc[pred == leaf].index
            statistics['precision_'+leaf][DAYS] = precision_score_ancestors(Tree, y_test.loc[leaf_ix], pred.loc[leaf_ix])
            statistics['recall_'+leaf][DAYS]    = recall_score_ancestors(Tree, y_test.loc[leaf_ix], pred.loc[leaf_ix])
            statistics['f1_'+leaf][DAYS]        = f1_score_ancestors(Tree, y_test.loc[leaf_ix], pred.loc[leaf_ix], beta = 1)
            
        print('DAY ',DAYS)
            
    return statistics

In [ ]:
# Run static HCOT baseline
stats = staticHierarchicalClassifier(0, 10)

#### Base Case 2 (Flat)

In [ ]:
def dynamicFlatClassifier(START, END):  

    # Optimal hyperparameters obtained via flat validation
    hypers = pd.DataFrame({'LR_penalty'     : ['l1','l1','l1','l1','l1','l1','l2','l1','l1','l1','l1'],
                           'RF_max_depth'   : [14,14,14,14,14,14,14,14,14,14,14], 
                           'RF_n_estimators': [45,45,40,45,40,45,40,45,45,45,45]})
    
    # Create hierarchy
    Tree = ClassHierarchy('ORDERS')
    Tree.add_node(['UNKNOWN','KNOWN'], 'ORDERS')
    Tree.add_node(['HAPPY','UNHAPPY'], 'KNOWN')
    Tree.add_node(['MILDLY UNHAPPY','MEDIUM UNHAPPY','HEAVILY UNHAPPY'], 'UNHAPPY')

    # Set certainty parameter alpha
    certainties = [0.7]
    
    statistics = {'accuracy'  :{},
                  'classified':{},
                  'thresholds':{},
                  'precision' :{},
                  'recall'    :{},
                  'f1'        :{}}
    for leaf in ['HAPPY','UNKNOWN','MILDLY UNHAPPY','MEDIUM UNHAPPY','HEAVILY UNHAPPY']: 
        statistics['precision_'+leaf] = {}
        statistics['recall_'+leaf]    = {}
        statistics['f1_'+leaf]        = {}
    for leaf in ['HAPPY','UNKNOWN','MILDLY UNHAPPY','MEDIUM UNHAPPY','HEAVILY UNHAPPY']: 
        statistics['2precision_'+leaf] = {}
        statistics['2recall_'+leaf]    = {}
        statistics['2f1_'+leaf]        = {}
    
    for CERTAINTY in certainties:  
        for DAYS in range(START, END+1):

            # Create X and Y
            X, y = dataX(df, DATE, X_col, Y_col, historic_variable, DAYS)

            # Make 80-20 train-test split and burn in first 10% of train set
            X_train_preburn = X.iloc[:int(0.8*len(X))]
            y_train_preburn = y.iloc[:int(0.8*len(y))]

            X_train = X_train_preburn.iloc[int(0.1*len(X_train_preburn)):]
            y_train = y_train_preburn.iloc[int(0.1*len(y_train_preburn)):]

            X_test = X.iloc[int(0.8*len(X)):]
            y_test = y.iloc[int(0.8*len(y)):]

            # Fit classifiers
            if DAYS < 5:
                clf = LogisticRegression(random_state=0, class_weight='balanced', solver = 'liblinear', penalty = hypers.loc[DAYS, 'LR_penalty'])
            else:
                clf = RandomForestClassifier(random_state=0, class_weight='balanced', max_depth = hypers.loc[DAYS, 'RF_max_depth'], n_estimators = hypers.loc[DAYS, 'RF_n_estimators'])
                
            clf.fit(X_train, y_train)

            y_train_hat = clf.predict_proba(X_train) 
            y_classes = clf.classes_
            y_train_hat = pd.DataFrame(y_train_hat, index = X_train.index, columns = y_classes)
            probs = pd.concat([y_train, y_train_hat], axis=1)
            
            # Compute thresholds
            THRESHOLDS = {}
            for node in range(1,6):
                name, threshold = flat_thresholds(probs, node, DAYS, CERTAINTY, steps = 100)
                THRESHOLDS[name] = threshold

            if DAYS == START: # create dataframe to save predictions
                y_hat = pd.DataFrame(['ORDERS'] * len(X_test),
                                        columns=[DAYS],
                                        index=X_test.index)
                index_no_leaf = X_test.index
            else:
                y_hat[DAYS] = y_hat[DAYS - 1]

            if DAYS < END:
                X_test_ = X_test.loc[index_no_leaf]
                y_proba = clf.predict_proba(X_test_)
                y_classes = clf.classes_
            
                max_prob = np.amax(y_proba, axis=1)              # max probability of classes
                max_class = np.argmax(y_proba, axis=1)           # class number with max probability
                max_class_thresholds = np.vectorize(lambda x: THRESHOLDS[y_classes[x]])(max_class)  # get node-specific threshold

                accept_index = np.where(max_prob >= max_class_thresholds)[0]
                accept_class = np.take(max_class, accept_index)  # filtered list of orders which are above threshold

                if len(accept_class) > 0: # check if samples reach threshold
                    accept_label = np.vectorize(lambda x: y_classes[x])(accept_class)                              # convert class number into label
                    y_hat_stage = pd.DataFrame(accept_label, index = np.take(X_test_.index.values, accept_index))  # set labels to correct position
                else:
                    y_hat_stage = pd.DataFrame(columns = [0], index = X_test_.index)
                    
                index_leaf = y_hat_stage.index

            else:
                pred        = clf.predict(X_test.loc[index_no_leaf]) # last day you want a label for each order
                y_hat_stage = pd.DataFrame(pred, index = index_no_leaf)
                index_leaf  = index_no_leaf

            y_hat = y_hat.assign(stage_col = y_hat_stage)
            y_hat.stage_col = y_hat.stage_col.fillna(y_hat[DAYS]) # fill previously predicted labels
            y_hat = y_hat.drop(DAYS, axis=1)
            y_hat = y_hat.rename(columns={'stage_col': DAYS})

            current_pred = y_hat.iloc[:, y_hat.shape[1] - 1]
            check_no_leaf = (current_pred == 'ORDERS')    # from current non_leaf predictions which are now leaf
            index_no_leaf = check_no_leaf[check_no_leaf].index
            
            # Compute performance metrics
            statistics['accuracy'][DAYS]   = metrics.accuracy_score(y_test.loc[index_leaf], current_pred.loc[index_leaf])
            statistics['classified'][DAYS] = (current_pred != 'ORDERS').sum() / len(y_test)
            statistics['thresholds'][DAYS] = THRESHOLDS
            
            precision, recall, f1, support = metrics.precision_recall_fscore_support(y_test.loc[index_leaf], current_pred.loc[index_leaf], average = 'weighted', beta = 1)
            
            statistics['precision'][DAYS] = precision
            statistics['recall'][DAYS]    = recall
            statistics['f1'][DAYS]        = f1
            
            precision, recall, f1, support = metrics.precision_recall_fscore_support(y_test.loc[index_leaf], current_pred.loc[index_leaf], average = None, labels = ['HAPPY','UNKNOWN','MILDLY UNHAPPY','MEDIUM UNHAPPY','HEAVILY UNHAPPY'])
            
            for ix,leaf in enumerate(['HAPPY','UNKNOWN','MILDLY UNHAPPY','MEDIUM UNHAPPY','HEAVILY UNHAPPY']):
                statistics['precision_'+leaf][DAYS] = precision[ix]
                statistics['recall_'+leaf][DAYS]    = recall[ix]
                statistics['f1_'+leaf][DAYS]        = f1[ix]
                
            for leaf in ['HAPPY','UNKNOWN','MILDLY UNHAPPY','MEDIUM UNHAPPY','HEAVILY UNHAPPY']:
                leaf_ix = current_pred.loc[current_pred == leaf].index
                statistics['2precision_'+leaf][DAYS] = precision_score_ancestors(Tree, y_test['detailedMatchClassification'].loc[leaf_ix], current_pred.loc[leaf_ix])
                statistics['2recall_'+leaf][DAYS]    = recall_score_ancestors(Tree, y_test['detailedMatchClassification'].loc[leaf_ix], current_pred.loc[leaf_ix])
                statistics['2f1_'+leaf][DAYS]        = f1_score_ancestors(Tree, y_test['detailedMatchClassification'].loc[leaf_ix], current_pred.loc[leaf_ix], beta = 1)
            
            # Write a path and file name (with .json format) to write the output to
            file_name = 'flat_statistics_'+str(CERTAINTY)+'.json'
            path_name = 'path...' + file_name
            with open(path_name, 'w') as f:
                json.dump(statistics, f, cls = NumpyEncoder)

            print('DAYS: ',DAYS)
     
        final_pred = y_hat.iloc[:, y_hat.shape[1] - 1]
        accuracy = metrics.accuracy_score(y_test, final_pred)
        precision, recall, f1, support = metrics.precision_recall_fscore_support(y_test, final_pred, average = None, labels = ['HAPPY','UNKNOWN','MILDLY UNHAPPY','MEDIUM UNHAPPY','HEAVILY UNHAPPY'], beta = 1)
        print(accuracy)
        print(precision)
        print(recall)
        print(precision_score_ancestors(Tree, y_test['detailedMatchClassification'], final_pred))
        print(recall_score_ancestors(Tree, y_test['detailedMatchClassification'], final_pred))
        for leaf in ['HAPPY','UNKNOWN','MILDLY UNHAPPY','MEDIUM UNHAPPY','HEAVILY UNHAPPY']:
            leaf_ix = final_pred.loc[final_pred == leaf].index
            print(leaf,precision_score_ancestors(Tree, y_test['detailedMatchClassification'].loc[leaf_ix], final_pred.loc[leaf_ix]),
                  recall_score_ancestors(Tree, y_test['detailedMatchClassification'].loc[leaf_ix], final_pred.loc[leaf_ix]))
        
    return final_pred, statistics

In [ ]:
# Run flat CAT-HCOT baseline
pred, stats = dynamicFlatClassifier(0, 10)

#### Functions

In [ ]:
def get_performance(DAYS, END, pred, current_pred, index_leaf, index_no_leaf, previous_pred_block, THRESHOLDS, OPTION, CERTAINTY, y_test, Tree, HC, feature_importances, statistics):
    
    # Initialize Dictionary at Day 0
    
    if DAYS == 0:
        statistics = {'%classified'     :{}, 'N_classified'         :{},  'N_predicted' : {},
                      'leaf_accuracy'   :{}, 'total_leaf_accuracy'  :{},
                      'leaf_precision'  :{}, 'total_leaf_precision' :{},
                      'leaf_recall'     :{}, 'total_leaf_recall'    :{},
                      'label_precision' :{}, 
                      'label_recall'    :{}, 
                      'block_precision' :{},
                      'block_recall'    :{},
                      'block_Nchange'   :{}, 'block_Pchange'        :{},
                      '%blocking'       :{}, '%Tblocking'           :{},
                      'tree_error'      :{},
                      'thresholds'      :{},
                      'option'          :{},
                      'certainty'       :{}}

        for leaf in Tree._get_leaf_nodes()+Tree._get_internal_nodes(): 
            statistics['precision_'+leaf] = {}
            statistics['recall_'+leaf]    = {}
            statistics['f1_'+leaf]        = {}
            
        feature_importances = pd.DataFrame(index = X_col)
        decision_trees = {}
    
    # Get Daily information
    
    check_block = pred.isin(Tree._get_internal_nodes())
    index_block = check_block[check_block].index        
        
    total_check_leaf = current_pred.isin(Tree._get_leaf_nodes())   # of all predictions which are now leaf
    total_index_leaf = total_check_leaf[total_check_leaf].index
        
    if DAYS > 0:
        block = pd.concat([previous_pred_block, pred.loc[previous_pred_block.index]], axis=1, keys = [0,1])
        block['Nchange'] = block.apply(lambda row: 0 if row[1] in Tree._get_descendants(row[0])+[row[0]] else 1, axis = 1)
        block['Pchange'] = block.apply(lambda row: 1 if row[1] in Tree._get_descendants(row[0]) else 0, axis = 1)
    previous_pred_block = pred.loc[index_block]
    previous_index_block = index_block #was commented?

    y_test = y_test['detailedMatchClassification']
    test_pred = pd.concat([y_test.loc[index_leaf], current_pred[index_leaf]], axis=1, keys = [0,1])
    test_pred['TE'] = test_pred.apply(lambda row: Tree._tree_distance(row[0], row[1]), axis = 1)
    
    # Update Dictionary
    
    statistics['option'][DAYS]          = OPTION
    statistics['certainty'][DAYS]       = CERTAINTY
    statistics['thresholds'][DAYS]      = THRESHOLDS
    statistics['%classified'][DAYS]     = current_pred.isin(Tree._get_leaf_nodes()).sum() / len(y_test)
    statistics['N_classified'][DAYS]    = int(len(index_leaf))
    statistics['N_predicted'][DAYS]     = int(len(pred))

    statistics['leaf_accuracy'][DAYS]   = metrics.accuracy_score(y_test.loc[index_leaf], pred.loc[index_leaf])
    statistics['leaf_precision'][DAYS]  = precision_score_ancestors(Tree, y_test.loc[index_leaf], pred.loc[index_leaf])
    statistics['leaf_recall'][DAYS]     = recall_score_ancestors(Tree, y_test.loc[index_leaf], pred.loc[index_leaf])

    for leaf in Tree._get_leaf_nodes()+Tree._get_internal_nodes():
        leaf_ix = pred.loc[pred == leaf].index
        statistics['precision_'+leaf][DAYS] = precision_score_ancestors(Tree, y_test.loc[leaf_ix], pred.loc[leaf_ix])
        statistics['recall_'+leaf][DAYS]    = recall_score_ancestors(Tree, y_test.loc[leaf_ix], pred.loc[leaf_ix])
        statistics['f1_'+leaf][DAYS]        = f1_score_ancestors(Tree, y_test.loc[leaf_ix], pred.loc[leaf_ix], beta = 1)
        
    for clf in list(HC.stages.keys()):
        if isinstance(HC.stages[clf]['classifier'],RandomForestClassifier):
            feature_importances[clf+'_'+str(DAYS)] = HC.stages[clf]['classifier'].feature_importances_ 
        elif isinstance(HC.stages[clf]['classifier'],LogisticRegression):
            feature_importances[clf+'_'+str(DAYS)] = HC.stages[clf]['classifier'].coef_[0] 

    statistics['total_leaf_accuracy'][DAYS]  = metrics.accuracy_score(y_test.loc[total_index_leaf], current_pred.loc[total_index_leaf])
    statistics['total_leaf_precision'][DAYS] = precision_score_ancestors(Tree, y_test.loc[total_index_leaf], current_pred.loc[total_index_leaf])
    statistics['total_leaf_recall'][DAYS]    = recall_score_ancestors(Tree, y_test.loc[total_index_leaf], current_pred.loc[total_index_leaf])

    statistics['label_precision'][DAYS]  = precision_score_ancestors(Tree, y_test.loc[index_leaf.union(index_block)], pred.loc[index_leaf.union(index_block)]) 
    statistics['label_recall'][DAYS]     = recall_score_ancestors(Tree, y_test.loc[index_leaf.union(index_block)], pred.loc[index_leaf.union(index_block)])  

    statistics['block_precision'][DAYS] = precision_score_ancestors(Tree, y_test.loc[index_block], pred.loc[index_block]) if DAYS < END else None
    statistics['block_recall'][DAYS]    = recall_score_ancestors(Tree, y_test.loc[index_block], pred.loc[index_block]) if DAYS < END else None
    statistics['block_Nchange'][DAYS]   = block['Nchange'].sum() / block['Nchange'].count() if DAYS > 0 else None
    statistics['block_Pchange'][DAYS]   = block['Pchange'].sum() / block['Pchange'].count() if DAYS > 0 else None
    statistics['%blocking'][DAYS]       = HC.blocking  if len(total_index_leaf) != len(y_test) else {'ORDERS':None,'KNOWN':None,'UNHAPPY':None}
    statistics['%Tblocking'][DAYS]      = HC.Tblocking if len(total_index_leaf) != len(y_test) else {'ORDERS':None,'KNOWN':None,'UNHAPPY':None}

    statistics['tree_error'][DAYS]      = np.mean(test_pred['TE'])
        
    return statistics, feature_importances, previous_pred_block

In [ ]:
def global_scores(y_true, y_pred, average = 'macro'):
    accuracy = metrics.accuracy_score(y_true, y_pred)
    scores = metrics.precision_recall_fscore_support(y_true, y_pred, average = average)
    return accuracy, scores[0], scores[1], scores[2]

def local_scores(y_true, y_pred):
    labels = np.unique(y_true)
    scores = metrics.precision_recall_fscore_support(y_true, y_pred, average = None, labels = labels, beta = 1)
    return scores[0], scores[1], scores[2]

def class_report(y_true, y_pred):
    print(metrics.classification_report(y_true, y_pred))

def _aggregate_class_sets(set_function, y_true, y_pred):
    intersection_sum = 0
    true_sum = 0
    predicted_sum = 0
    for true, pred in zip(list(y_true), list(y_pred)):
        true_set = set([true] + set_function(true))
        pred_set = set([pred] + set_function(pred))
        intersection_sum += len(true_set.intersection(pred_set))
        true_sum += len(true_set)
        predicted_sum += len(pred_set)
    return (true_sum, predicted_sum, intersection_sum)

def precision_score_ancestors(class_hierarchy, y_true, y_pred):
    true_sum, predicted_sum, intersection_sum = _aggregate_class_sets(
        class_hierarchy._get_ancestors, y_true, y_pred)
    if predicted_sum == 0:
        return None
    else:
        return intersection_sum / predicted_sum

def recall_score_ancestors(class_hierarchy, y_true, y_pred):
    true_sum, predicted_sum, intersection_sum = _aggregate_class_sets(
        class_hierarchy._get_ancestors, y_true, y_pred)
    if true_sum == 0:
        return None
    else:
        return intersection_sum / true_sum

def f1_score_ancestors(class_hierarchy, y_true, y_pred, beta):
    precision = precision_score_ancestors(class_hierarchy, y_true, y_pred)
    recall = recall_score_ancestors(class_hierarchy, y_true, y_pred)
    if (precision == None) or (recall == None):
        return None
    elif (precision == 0) or (recall == 0):
        return 0
    else:
        return ((beta ** 2 + 1) * precision * recall) / ((beta ** 2 * precision) + recall)

In [ ]:
class ClassHierarchy:
    
    def __init__(self, root):
        self.root = root
        self.nodes = {}
        
    def add_node(self, children, parent):
        for child in children:
            self.nodes[child] = parent
            
    def _get_leaf_nodes(self):
        leaf_nodes = []
        for child in self.nodes.keys():
            if self._get_children(child) == []:
                leaf_nodes.append(child)
        return leaf_nodes
    
    def _get_internal_nodes(self):
        internal_nodes = []
        leaves = self._get_leaf_nodes()
        for child in self.nodes.keys():
            if (child != self.root) and (child not in leaves):
                internal_nodes.append(child)
        return internal_nodes

    def _get_children(self, parent):
        return sorted([child for child, childs_parent in
                       self.nodes.items() if childs_parent == parent])
    
    def _get_parent(self, child):
        return self.nodes[child] if (child in self.nodes and child != self.root) else self.root
    
    def _get_ancestors(self, child):
        # Not including root, not including the child
        ancestors = []
        while True:
            child = self._get_parent(child)
            if child == self.root:
                break
            ancestors.append(child)
        return ancestors
    
    def _get_descendants(self, parent):
        # Return a list of the descendants of this node, not including the parent
        descendants = []
        self._depth_first(parent, descendants)
        descendants.remove(parent)
        return descendants
    
    def _depth_first(self, parent, classes):
        classes.append(parent)
        for node in self._get_children(parent):
            self._depth_first(node, classes)
            
    def _tree_distance(self, y_test, pred):
        
        y_test_path = [y_test] + self._get_ancestors(y_test) + [self.root] if y_test != self.root else [y_test] + self._get_ancestors(y_test)
        pred_path   = [pred] + self._get_ancestors(pred) + [self.root] if pred != self.root else [pred] + self._get_ancestors(pred)
        
        y_test_edges = []
        for ix, node in enumerate(y_test_path):
            length = len(y_test_path)
            if ix < length - 1:
                y_test_edges.append((node, y_test_path[ix+1]))
                
        pred_edges = []
        for ix, node in enumerate(pred_path):
            length = len(pred_path)
            if ix < length - 1:
                pred_edges.append((node, pred_path[ix+1]))        
        
        tree_distance = len([edge for edge in y_test_edges + pred_edges if edge not in pred_edges or edge not in y_test_edges])
        
        return tree_distance

In [ ]:
class HierarchicalClassifier:

    def __init__(self, class_hierarchy):
        self.stages = {}
        self.class_hierarchy = class_hierarchy
        self._create_stages(self.stages, self.class_hierarchy.root, 0)

    def _create_stages(self, stages, parent, depth):
        # Get the children of this parent
        children = self.class_hierarchy._get_children(parent)
        
        if len(children) > 0:
            stage = {}
            stage['depth'] = depth
            stage['labels'] = children
            stage['classes'] = stage['labels'] + [parent]
            stage['target'] = 'target_stage_' + parent
            stages[parent] = stage

            for node in children:
                self._create_stages(stages, node, depth + 1)
                
    def _recode_label(self, classes, label):

        while label != self.class_hierarchy.root and label not in classes:
            label = self.class_hierarchy._get_parent(label)
        return label
                
    def _prep_data(self, X, y):
        
        Xcols = range(0, X.shape[1])
        Ycol = X.shape[1]
        
        df = pd.concat([X, y], axis=1, ignore_index=True)
        # Create a target column for each stage with the recoded labels
        for stage_name, stage_info in self.stages.items():
            df[stage_info['target']] = pd.DataFrame.apply(df[[Ycol]],
                                    lambda row: self._recode_label(stage_info['classes'], row[Ycol]),
                                    axis=1)
        return df, Xcols
    
    def _label_mapping(self, y_train, stage_name):
        labels = np.unique(y_train)
        int_label_mapping = dict(enumerate(labels))
        label_int_mapping = {y:x for x,y in int_label_mapping.items()}
        self.stages[stage_name]['mapping'] = {'int_label':int_label_mapping,
                                              'label_int':label_int_mapping}
        
    def _class_weights(self, y_train, stage_name):
        class_weights = class_weight.compute_class_weight('balanced',classes = np.unique(y_train),y = y_train)
        class_weights = dict(enumerate(class_weights))
        self.stages[stage_name]['classifier'].set_params(class_weight = class_weights)
    
    def fit_classifiers(self, classifiers):
        """
        Fit a classifier to each stage
        """
        if classifiers.keys() != self.stages.keys():
             raise ValueError('Your assigned classifiers do not match the stages of the hierarchy, fit a classifier to each of: '+self.stages.keys())
        else:
            for stage, classifier in classifiers.items():
                self.stages[stage]['classifier'] = classifier
    
    def fit(self, X, y):
        """
        Build a multi-classifier from training data (X, y).
        """
        df, Xcols = self._prep_data(X, y)
        self.scaler = preprocessing.MinMaxScaler().fit(X)
        
        for stage_name, stage_info in self.stages.items():
            
            dfFilter = df[df[stage_info['target']].isin(stage_info['classes'])]
            
            X_train = dfFilter[Xcols]
            y_train = dfFilter[[stage_info['target']]]
                        
            # warning - no samples to fit for stage
            if isinstance(stage_info['classifier'], KerasClassifier):
                y_train_col = pd.Series(np.ravel(y_train))
                
                self._class_weights(y_train_col, stage_name)
                self._label_mapping(y_train_col, stage_name)

                y_encoded = y_train_col.map(stage_info['mapping']['label_int'])

                if len(stage_info['labels']) > 2:
                    y_dummy = pd.DataFrame(np_utils.to_categorical(y_encoded))
                    y_train_NN = y_dummy
                else:
                    y_train_NN = np.asarray(y_encoded).reshape((-1,1))

                X_scaled = pd.DataFrame(self.scaler.transform(X_train))
                stage_info['classifier'].fit(X_scaled, y_train_NN)
            else:
                stage_info['classifier'] = stage_info['classifier'].fit(X_train, y_train)
            # print('Stage '+stage_name+' succesfully fitted')

        return self
    
    def predict(self, X):
        
        stage_number = 0
        for stage_name, stage_info in self.stages.items():
            
            if stage_name == self.class_hierarchy.root:
                y_hat = pd.DataFrame([self.class_hierarchy.root] * len(X),
                                        columns=[self.class_hierarchy.root],
                                        index=X.index)
            else:
                y_hat[stage_name] = y_hat[list(self.stages.keys())[stage_number - 1]]
            stage_number += 1             
                
            X_test = X[y_hat[stage_name].isin([stage_name])]  # warning - no samples to fit for stage
            
            if X_test.empty:
                continue
            
            if isinstance(stage_info['classifier'], KerasClassifier):
                X_scaled = pd.DataFrame(self.scaler.transform(X_test))
                if len(stage_info['labels']) == 2:
                    y_pred = pd.Series(stage_info['classifier'].predict(X_scaled).flatten()).map(stage_info['mapping']['int_label'])
                else:
                    y_pred = pd.Series(stage_info['classifier'].predict(X_scaled)).map(stage_info['mapping']['int_label'])
                y_hat_stage = pd.DataFrame(y_pred.values, index = X_test.index)
            else:
                y_hat_stage = pd.DataFrame(stage_info['classifier'].predict(X_test), index = X_test.index)
                
            y_hat = y_hat.assign(stage_col = y_hat_stage)
            y_hat.stage_col = y_hat.stage_col.fillna(y_hat[stage_name]) #fill previously predicted labels
            y_hat = y_hat.drop(stage_name, axis=1)
            y_hat = y_hat.rename(columns={'stage_col': stage_name})
            
        return y_hat.iloc[:, y_hat.shape[1] - 1]     
    
    def predict_proba(self, X, threshold = 0.5):
        
        self.blocking = {}
        stage_number = 0
        for stage_name, stage_info in self.stages.items():
            
            if stage_name == self.class_hierarchy.root:
                y_hat = pd.DataFrame([self.class_hierarchy.root] * len(X),
                                        columns=[self.class_hierarchy.root],
                                        index=X.index)
            else:
                y_hat[stage_name] = y_hat[list(self.stages.keys())[stage_number - 1]]
            stage_number += 1             
                
            X_test = X[y_hat[stage_name].isin([stage_name])]  # warning - no samples to fit for stage
            
            if isinstance(stage_info['classifier'], KerasClassifier):
                X_scaled = pd.DataFrame(self.scaler.transform(X_test))
                y_proba = stage_info['classifier'].predict_proba(X_scaled)
                y_classes = list(stage_info['mapping']['int_label'].values())
            else:
                y_proba = stage_info['classifier'].predict_proba(X_test)
                y_classes = stage_info['classifier'].classes_
            
            max_prob = np.amax(y_proba, axis=1)              # max probability of classes
            max_class = np.argmax(y_proba, axis=1)           # class number with max probability
            accept_index = np.where(max_prob >= threshold)[0]# indexes which are above threshold
            accept_class = np.take(max_class, accept_index)  # filtered list of orders which are above threshold
            
            if len(accept_class) > 0: # check if samples reach threshold
                accept_label = np.vectorize(lambda x: y_classes[x])(accept_class)                             # convert class number into label
                y_hat_stage = pd.DataFrame(accept_label, index = np.take(X_test.index.values, accept_index))  # set labels to correct position
                self.blocking[stage_name] = 1 - (len(accept_class) / len(max_class)) # blocking factor
            else:
                y_hat_stage = pd.DataFrame(columns = [0], index = X_test.index)
                self.blocking[stage_name] = 1
                
            y_hat = y_hat.assign(stage_col = y_hat_stage)
            y_hat.stage_col = y_hat.stage_col.fillna(y_hat[stage_name]) # fill previously predicted labels
            y_hat = y_hat.drop(stage_name, axis=1)
            y_hat = y_hat.rename(columns={'stage_col': stage_name})
            
        return y_hat.iloc[:, y_hat.shape[1] - 1]
    
    def predict_proba2(self, X, THRESHOLDS):
        
        self.blocking = {}
        self.Tblocking = {}
        stage_number = 0
        for stage_name, stage_info in self.stages.items():
            
            if stage_name == self.class_hierarchy.root:
                y_hat = pd.DataFrame([self.class_hierarchy.root] * len(X),
                                        columns=[self.class_hierarchy.root],
                                        index=X.index)
            else:
                y_hat[stage_name] = y_hat[list(self.stages.keys())[stage_number - 1]]
            stage_number += 1             
                
            X_test = X[y_hat[stage_name].isin([stage_name])]  # warning - no samples to fit for stage
            
            if X_test.empty:
                self.blocking[stage_name] = None
                self.Tblocking[stage_name] = None
                continue
            
            if isinstance(stage_info['classifier'], KerasClassifier):
                X_scaled = pd.DataFrame(self.scaler.transform(X_test))
                y_proba = stage_info['classifier'].predict_proba(X_scaled)
                y_classes = list(stage_info['mapping']['int_label'].values())
            else:
                y_proba = stage_info['classifier'].predict_proba(X_test)
                y_classes = stage_info['classifier'].classes_
            
            max_prob = np.amax(y_proba, axis=1)              # max probability of classes
            max_class = np.argmax(y_proba, axis=1)           # class number with max probability
            max_class_thresholds = np.vectorize(lambda x: THRESHOLDS[y_classes[x]])(max_class)  # get node specific threshold
            
            accept_index = np.where(max_prob >= max_class_thresholds)[0]

            accept_class = np.take(max_class, accept_index)  # filtered list of orders which are above threshold
            
            if len(accept_class) > 0: # check if samples reach threshold
                accept_label = np.vectorize(lambda x: y_classes[x])(accept_class)                             # convert class number into label
                y_hat_stage = pd.DataFrame(accept_label, index = np.take(X_test.index.values, accept_index))  # set labels to correct position
                
                self.blocking[stage_name] = 1 - (len(accept_class) / len(max_class)) # blocking factor
                self.Tblocking[stage_name] = len(max_class) - len(accept_class)
            else:
                y_hat_stage = pd.DataFrame(columns = [0], index = X_test.index)
                self.blocking[stage_name] = 1
                self.Tblocking[stage_name] = len(max_class)
                
            y_hat = y_hat.assign(stage_col = y_hat_stage)
            y_hat.stage_col = y_hat.stage_col.fillna(y_hat[stage_name]) # fill previously predicted labels
            y_hat = y_hat.drop(stage_name, axis=1)
            y_hat = y_hat.rename(columns={'stage_col': stage_name})
            
        return y_hat.iloc[:, y_hat.shape[1] - 1]
    
    def get_probabilities(self, X, y):
        
        df, Xcols = self._prep_data(X, y)
        
        stage_number = 0
        
        y_hat = pd.DataFrame(columns = [self.class_hierarchy.root], index = X.index)
        
        for stage_name, stage_info in self.stages.items():
                
            stage_number += 1             
            
            dfFilter = df[df[stage_info['target']].isin(stage_info['classes'])]
            
            X_test = dfFilter[Xcols]
            y_test = dfFilter[[stage_info['target']]]
            
            if isinstance(stage_info['classifier'], KerasClassifier):
                X_scaled = pd.DataFrame(self.scaler.transform(X_test))
                y_proba = stage_info['classifier'].predict_proba(X_scaled)
                y_classes = list(stage_info['mapping']['int_label'].values())
            else:
                y_proba = stage_info['classifier'].predict_proba(X_test)
                y_classes = stage_info['classifier'].classes_
            
            y_hat_stage = pd.DataFrame(y_proba, index = X_test.index)

            for col, label in enumerate(y_classes):
                y_hat[label] = y_hat_stage[col]
               
        return y_hat

In [ ]:
def get_probs(day):
    HC = HierarchicalClassifier(ch)
    HC.fit_classifiers({'ORDERS'  : DecisionTreeClassifier(random_state=0, class_weight='balanced', criterion = hypers.loc[day, '1_criterion'], max_depth = hypers.loc[day, '1_max_depth']),
                        'KNOWN'   : DecisionTreeClassifier(random_state=0, class_weight='balanced', criterion = hypers.loc[day, '2_criterion'], max_depth = hypers.loc[day, '2_max_depth']),
                        'UNHAPPY' : RandomForestClassifier(random_state=0, class_weight='balanced', max_depth = hypers.loc[day, '3_max_depth'], n_estimators = hypers.loc[day, '3_n_estimators'])})
    
    X, y  = functions.dataX(df_, DATE, X_col, Y_col, historic_variable, day)
    index = range(0, X.shape[0])

    X_train, X_test, y_train, y_test, ix_train, ix_test = train_test_split(X, y, index, test_size=0.2, random_state=0, shuffle=False)

    HC.fit(X_train,y_train)
    y_hat = HC.get_probabilities(X_train, y_train)

    probs = pd.concat([y_train, y_hat], axis=1)
    
    return(probs)

def opt_threshold(probs, node, day, certainty, option, steps = 100):
    
    if node == 1:
        probabilities_for = 'UNKNOWN'
        y_pos_filter_list = ['UNKNOWN']
        y_neg_filter_list = ['HAPPY', 'MILDLY UNHAPPY', 'MEDIUM UNHAPPY', 'HEAVILY UNHAPPY']
        level             = probs[['UNKNOWN', 'KNOWN']]
        majority_vote     = level[level['UNKNOWN'] > level['KNOWN']]['UNKNOWN']
    elif node == 2:
        probabilities_for = 'KNOWN'
        y_pos_filter_list = ['HAPPY', 'MILDLY UNHAPPY', 'MEDIUM UNHAPPY', 'HEAVILY UNHAPPY']
        y_neg_filter_list = ['UNKNOWN']
        level             = probs[['UNKNOWN', 'KNOWN']]
        majority_vote     = level[level['KNOWN'] > level['UNKNOWN']]['KNOWN']
    elif node == 3:
        probabilities_for = 'HAPPY'
        y_pos_filter_list = ['HAPPY']
        y_neg_filter_list = ['MILDLY UNHAPPY', 'MEDIUM UNHAPPY', 'HEAVILY UNHAPPY']
        level             = probs[probs.detailedMatchClassification.isin(['MILDLY UNHAPPY', 'MEDIUM UNHAPPY', 'HEAVILY UNHAPPY', 'HAPPY'])][['UNHAPPY', 'HAPPY']]
        majority_vote     = level[level['HAPPY'] > level['UNHAPPY']]['HAPPY']
    elif node == 4:
        probabilities_for = 'UNHAPPY'
        y_pos_filter_list = ['MILDLY UNHAPPY', 'MEDIUM UNHAPPY', 'HEAVILY UNHAPPY']
        y_neg_filter_list = ['HAPPY']
        level             = probs[probs.detailedMatchClassification.isin(['MILDLY UNHAPPY', 'MEDIUM UNHAPPY', 'HEAVILY UNHAPPY', 'HAPPY'])][['UNHAPPY', 'HAPPY']]
        majority_vote     = level[level['UNHAPPY'] > level['HAPPY']]['UNHAPPY']
    elif node == 5:
        probabilities_for = 'MILDLY UNHAPPY'
        y_pos_filter_list = ['MILDLY UNHAPPY']
        y_neg_filter_list = ['MEDIUM UNHAPPY', 'HEAVILY UNHAPPY']
        level             = probs[probs.detailedMatchClassification.isin(['MILDLY UNHAPPY', 'MEDIUM UNHAPPY', 'HEAVILY UNHAPPY'])][['MILDLY UNHAPPY', 'MEDIUM UNHAPPY', 'HEAVILY UNHAPPY']]
        majority_vote     = level[ (level['MILDLY UNHAPPY'] > level['MEDIUM UNHAPPY']) & (level['MILDLY UNHAPPY'] > level['HEAVILY UNHAPPY']) ]['MILDLY UNHAPPY']
    elif node == 6:
        probabilities_for = 'MEDIUM UNHAPPY'
        y_pos_filter_list = ['MEDIUM UNHAPPY']
        y_neg_filter_list = ['MILDLY UNHAPPY', 'HEAVILY UNHAPPY']
        level             = probs[probs.detailedMatchClassification.isin(['MILDLY UNHAPPY', 'MEDIUM UNHAPPY', 'HEAVILY UNHAPPY'])][['MILDLY UNHAPPY', 'MEDIUM UNHAPPY', 'HEAVILY UNHAPPY']]
        majority_vote     = level[ (level['MEDIUM UNHAPPY'] > level['MILDLY UNHAPPY']) & (level['MEDIUM UNHAPPY'] > level['HEAVILY UNHAPPY']) ]['MEDIUM UNHAPPY']
    elif node == 7:
        probabilities_for = 'HEAVILY UNHAPPY'
        y_pos_filter_list = ['HEAVILY UNHAPPY']
        y_neg_filter_list = ['MILDLY UNHAPPY', 'MEDIUM UNHAPPY']
        level             = probs[probs.detailedMatchClassification.isin(['MILDLY UNHAPPY', 'MEDIUM UNHAPPY', 'HEAVILY UNHAPPY'])][['MILDLY UNHAPPY', 'MEDIUM UNHAPPY', 'HEAVILY UNHAPPY']]
        majority_vote     = level[ (level['HEAVILY UNHAPPY'] > level['MEDIUM UNHAPPY']) & (level['HEAVILY UNHAPPY'] > level['MILDLY UNHAPPY']) ]['HEAVILY UNHAPPY']
    else:
        raise Exception('''Error: undefined node has been passed. Node options (integer input):
                           1: Unknown
                           2: Known
                           3: Happy
                           4: Unhappy
                           5: Mildly Unhappy
                           6: Medium Unhappy
                           7: Heavily Unhappy''')
    
    y_pos = probs[probs.detailedMatchClassification.isin(y_pos_filter_list)][probabilities_for] 
    y_neg = probs[probs.detailedMatchClassification.isin(y_neg_filter_list)][probabilities_for]
    
    if option == 1:
        y_pos = y_pos[y_pos > min(majority_vote)]
        y_neg = y_neg[y_neg > min(majority_vote)]
    elif option == 2:
        y_pos = y_pos[y_pos.index.isin(majority_vote.index)]
        y_neg = y_neg[y_neg.index.isin(majority_vote.index)]
    else:
        raise Exception('''Error: undefined threshold option has been passed. Threshold options (integer input):
                           1: Consider all probabilities >= min(majority vote)
                           2: Only consider probabilities that are the majority vote''')
    
    # Potential thresholds
    V = np.concatenate((y_pos, y_neg))
    V = np.unique(V) # np.unique() also sorts
    
    if len(y_neg) > 0:
        lowerbound = np.percentile(y_neg, (certainty*100))
    else:
        lowerbound = V.min()
    
    V = V[V >= lowerbound] # define allowed search space
    
    S = np.linspace(V.min(), V.max(), steps)
    
    thresholds = pd.DataFrame({'threshold'     : [0]*steps,
                               'F_score'       : [0]*steps})
    
    for i in range(steps):        
        threshold = S[i]       
        beta      = 1
        positives = len(y_pos[y_pos >= threshold])  
        negatives = len(y_neg[y_neg >= threshold]) 
        recall    = positives / len(y_pos)
        precision = positives / (positives + negatives)

        thresholds.loc[i, 'threshold']       = threshold
        thresholds.loc[i, 'F_score']         = ((beta ** 2 + 1) * precision * recall) / ((beta ** 2 * precision) + recall) if ((beta ** 2 * precision) + recall) != 0 else 0
        
    F_score         = thresholds['F_score'].max()
    opt_index       = thresholds['F_score'].argmax()
    threshold       = thresholds.loc[opt_index, 'threshold']
    
    return(probabilities_for, threshold)

def flat_thresholds(probs, node, day, certainty, steps = 100):
    
    NODES = set(['UNKNOWN', 'HAPPY', 'MILDLY UNHAPPY', 'MEDIUM UNHAPPY', 'HEAVILY UNHAPPY'])
    
    if node == 1:   NODE = 'UNKNOWN'
    elif node == 2: NODE = 'HAPPY'
    elif node == 3: NODE = 'MILDLY UNHAPPY'
    elif node == 4: NODE = 'MEDIUM UNHAPPY'
    elif node == 5: NODE = 'HEAVILY UNHAPPY'
    else:
        raise Exception('''Error: undefined node has been passed. Node options (integer input):
                           1: Unknown
                           2: Happy
                           3: Mildly Unhappy
                           4: Medium Unhappy
                           5: Heavily Unhappy''')
        
    probabilities_for = NODE
    y_pos_filter_list = [NODE]
    y_neg_filter_list = list(NODES - {NODE})
    majority_vote     = probs[probs[NODE] > probs[list(NODES - {NODE})].max(axis=1)][NODE]
        
    y_pos = probs[probs.detailedMatchClassification.isin(y_pos_filter_list)][probabilities_for] 
    y_neg = probs[probs.detailedMatchClassification.isin(y_neg_filter_list)][probabilities_for]
    
    y_pos = y_pos[y_pos.index.isin(majority_vote.index)]
    y_neg = y_neg[y_neg.index.isin(majority_vote.index)]
 
    if len(y_neg) > 0:
        lowerbound = np.percentile(y_neg, (certainty*100))
    else:
        lowerbound = V.min()
    
    # Potential thresholds
    V = np.concatenate((y_pos, y_neg))
    V = np.unique(V) # np.unique() also sorts    
    V = V[V >= lowerbound] # define allowed search space
    S = np.linspace(V.min(), V.max(), steps)
    
    thresholds = pd.DataFrame({'threshold'     : [0]*steps,
                               'F_score'       : [0]*steps})
    
    for i in range(steps):        
        threshold = S[i] 
        beta      = 1
        positives = len(y_pos[y_pos >= threshold])
        negatives = len(y_neg[y_neg >= threshold])
        recall    = positives / len(y_pos)
        precision = positives / (positives + negatives)

        thresholds.loc[i, 'threshold']       = threshold
        thresholds.loc[i, 'F_score']         = ( (1 + (beta**2)) * precision * recall ) / ( (beta**2) * precision + recall ) if ( (beta**2) * precision + recall ) != 0 else 0
        
    
    F_score         = thresholds['F_score'].max()
    opt_index       = thresholds['F_score'].argmax()
    threshold       = thresholds.loc[opt_index, 'threshold']                           

    return(probabilities_for, threshold)